## Discriminating sub graphes
Now that the devices have been found we have to automatically discriminate between the interesting devices and least interesting ones,  
we will do that using k-means clustering and clustering on sub graph length and total distance accrued to create two sets of two cluster,  
Short vs long subgraphes, mobile vs non mobile devices

In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.cluster import KMeans
import networkx as nx

import sys  
sys.path.insert(0, './Functions/')

from Object import Position_graph
import Subgraph

In [2]:
%%bigquery df_onroute_position_window
Select *
from data-prod-270222.datascience.position_raw

Downloading: 100%|██████████| 82228/82228 [00:00<00:00, 152374.86rows/s]


In [3]:
mygraph = Position_graph(df_onroute_position_window)
mygraph.window_kwargs = {
            'min_window_len':10,
            'min_value_len':5,
        }
mygraph.compute_window()
mygraph.compute_graph(seq_pourc = 0.25)
#adding time window attribute to graph
#Subgraph.set_nodes_time_window_attribute(mygraph, mygraph.graph.nodes)

mygraph.compute_subgraphs(pourcentage_complete_paths = 0.25)

New empty graph
Length of dataset: 82228
Slice from 0 to 128 computed for distance stored under the active_distance_slice attribute


100%|██████████| 128/128 [01:13<00:00,  1.75it/s]


Computed 191 roots with a total of 2583 total paths!
    Found 43 device!
Computed 132 roots with a total of 1460 total paths!
    Found 15 device!
Computed 83 roots with a total of 1437 total paths!
    Found 12 device!
Added 70 devices subgraphs under the subgraph attribute


In [4]:
mygraph.subgraph.sort(key=lambda x: x[1][0])

In [5]:
#removing the smallish subgraph in favor for the bigger ones and clustering on only distance travelled
devices_to_cluster = [(d[-1][0],1) for d in mygraph.subgraph if d[-1][1]>3]
#separating data in 2 cluster, hopefully non mobile and mobile devices!
kmeans = KMeans(n_clusters=2).fit_predict(devices_to_cluster)
devices_clustered = np.insert([d for d in mygraph.subgraph if d[-1][1]>3], 0, kmeans, axis=1)

if np.mean([d[-1][0] for d in devices_clustered if d[0]==0])>np.mean([d[-1][0] for d in devices_clustered if d[0]==1]):
    mobile_cluster_index = 0
else:
    mobile_cluster_index = 1

/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:4663: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [6]:
mobile_devices = [d for d in devices_clustered if d[0]==mobile_cluster_index]